<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #5: TF-IDF
`Fecha de entrega: Marzo 25, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [37]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

# Punto 1: Pre-Procesamiento

- `[9 pts]` Leer el archivo `bob_esponja.csv` usando `pandas`
- `[9 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [38]:
data = pd.read_csv("C:/Users/dcard/OneDrive/Documentos/Procesamiento_del_lenguaje_Natural/bob_esponja.csv", index_col=0)
data

,Descripción
Personaje,
Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...
Patricio Estrella,"Es una estrella de mar de color rosa, el mejor..."
Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...
Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...
Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...
Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e..."
Karen Plankton,Karen Plankton es uno de los dos principales a...
Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ..."
Sra. Puff,La Señora Puff es un personaje principal de Bo...


In [66]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    texto = " ".join(texto)
    return texto


def textoEst(texto):
    texto_stem = []
    #texto = texto.to_string()
    texto = texto.split()
    for palabra in texto:
        raiz = spanishStemmer.stem(palabra)
        texto_stem.append(raiz)
    return texto_stem

data['pre-procesado'] = data['Descripción'].apply(lambda texto: pre_procesado(texto))
#data['pre-procesado']

#textoEst(data['pre-procesado'])
data['datastemming'] = data['pre-procesado'].apply(lambda texto: textoEst(texto))
data1 = data['datastemming'].to_string()

# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vec = TfidfVectorizer()
tfidf = tfidf_vec.fit_transform(data['pre-procesado'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray())
tfidf_matrix.columns = tfidf_vec.get_feature_names()
tfidf_matrix.index = data.index

tfidf_matrix = tfidf_matrix.T.round(3)

tfidf_matrix

Personaje,Bob Esponja Pantalones Cuadrados,Patricio Estrella,Calamardo Tentáculos,Arenita Mejillas,Eugenio H. Cangrejo,Sheldon J. Plankton,Karen Plankton,Perlita Cangrejo,Sra. Puff,Gary el Caracol
abajo,0.059,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000
aborrece,0.000,0.0,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
acaba,0.000,0.0,0.091,0.000,0.000,0.0,0.000,0.0,0.000,0.000
accidente,0.050,0.0,0.000,0.000,0.046,0.0,0.000,0.0,0.000,0.000
aceptó,0.000,0.0,0.000,0.000,0.000,0.0,0.064,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
zapatos,0.044,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.092,0.069
árbol,0.000,0.0,0.000,0.081,0.000,0.0,0.000,0.0,0.000,0.000
éste,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.093
única,0.059,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de los personajes
- `[2 pts]` ¿Cuáles son los personajes más parecidos?
- `[2 pts]` ¿Cuáles son los personajes más diferentes?

In [71]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

Personaje,Bob Esponja Pantalones Cuadrados,Patricio Estrella,Calamardo Tentáculos,Arenita Mejillas,Eugenio H. Cangrejo,Sheldon J. Plankton,Karen Plankton,Perlita Cangrejo,Sra. Puff,Gary el Caracol
Personaje,,,,,,,,,,
Bob Esponja Pantalones Cuadrados,0.000000,0.850696,0.927102,0.872229,0.881327,0.901863,0.957853,0.915862,0.882557,0.931151
Patricio Estrella,0.850696,0.000000,0.809173,0.893461,0.893984,0.968212,0.961591,0.936322,0.954731,0.922892
Calamardo Tentáculos,0.927102,0.809173,0.000000,0.944294,0.873432,0.921283,0.969339,0.972793,0.968218,0.914859
Arenita Mejillas,0.872229,0.893461,0.944294,0.000000,0.927194,0.982124,0.961457,0.982504,0.983133,0.930661
Eugenio H. Cangrejo,0.881327,0.893984,0.873432,0.927194,0.000000,0.705246,0.903650,0.829501,0.938176,0.928466
Sheldon J. Plankton,0.901863,0.968212,0.921283,0.982124,0.705246,0.000000,0.808338,0.921415,0.990023,0.971529
Karen Plankton,0.957853,0.961591,0.969339,0.961457,0.903650,0.808338,0.000000,0.984117,0.991405,0.978311
Perlita Cangrejo,0.915862,0.936322,0.972793,0.982504,0.829501,0.921415,0.984117,0.000000,0.895047,0.967758
Sra. Puff,0.882557,0.954731,0.968218,0.983133,0.938176,0.990023,0.991405,0.895047,0.000000,0.963548


# ¿Cuáles son los personajes más parecidos?

In [72]:
from sklearn.metrics.pairwise import euclidean_distances

dist_euc = euclidean_distances(tfidf_matrix.T.values)
dist_euc = pd.DataFrame(dist_euc, columns = tfidf_matrix.columns, index=tfidf_matrix.columns)

print(dist_euc)

Personaje                         Bob Esponja Pantalones Cuadrados  \
Personaje                                                            
Bob Esponja Pantalones Cuadrados                          0.000000   
Patricio Estrella                                         1.306876   
Calamardo Tentáculos                                      1.363981   
Arenita Mejillas                                          1.322932   
Eugenio H. Cangrejo                                       1.331643   
Sheldon J. Plankton                                       1.344001   
Karen Plankton                                            1.385879   
Perlita Cangrejo                                          1.353716   
Sra. Puff                                                 1.329562   
Gary el Caracol                                           1.366846   

Personaje                         Patricio Estrella  Calamardo Tentáculos  \
Personaje                                                                   
Bob E

# ¿Cuáles son los personajes más diferentes?

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

dist_euc = euclidean_distances(tfidf_matrix.T.values)
dist_euc = pd.DataFrame(dist_euc, columns = tfidf_matrix.columns, index=tfidf_matrix.columns)

print(dist_euc)